In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from src.utils.filter_conditions_monthly import filter_conditions_monthly

EUROSTAT_PATH = "src/data/raw/eurostat/latest_data.csv"
BNETZA_PATH = "src/data/raw/germany_household/latest_data.csv"
CBS_PATH = "src/data/raw/CBS_dutch_power.csv"

In [47]:
# 1. Load and preprocess daily demand data
df = pd.read_csv("src/data/processed/daily_demand_all.csv")
df['date'] = pd.to_datetime(df['date'].astype(str).str[:10])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Group by relevant columns and convert to TWh
df = df.groupby(['country', 'type', 'source', 'year', 'month'], as_index=False).agg({'demand': 'sum'})
df['demand'] = df['demand'] / 1000000000

In [48]:
# 2. Load and process Eurostat data
eurostat_df = pd.read_csv(EUROSTAT_PATH)
eurostat_df['date'] = pd.to_datetime(eurostat_df['date'])
eurostat_df['month'] = eurostat_df['date'].dt.month
eurostat_df['year'] = eurostat_df['date'].dt.year
del eurostat_df['date']
df = pd.concat([df, eurostat_df], ignore_index=True)

In [49]:
# 3. Load and process BNetzA data 
bnetza_df = pd.read_csv(BNETZA_PATH)
bnetza_df['date'] = pd.to_datetime(bnetza_df['date'])
bnetza_df['month'] = bnetza_df['date'].dt.month
bnetza_df['year'] = bnetza_df['date'].dt.year
bnetza_append = bnetza_df[['country', 'type', 'source', 'year', 'month', 'demand']]
df = pd.concat([df, bnetza_append], ignore_index=True)

In [50]:
# 4. Load and process CBS data
# we use historic CBS power data (pre July 2022) for power values and converting industry-power to industry
cbs_df = pd.read_csv(CBS_PATH)

#set the month and year columns as datetypes for each
cbs_df['month'] = pd.to_datetime(cbs_df['month'], format="%m").dt.month
cbs_df['year'] = pd.to_datetime(cbs_df['year'], format="%Y").dt.year

#isolate NL and 'industry-power' rows
df_nl_temp = df[(df['country'] == 'NL') & (df['type'] == 'industry')].copy()

#adjust older industry-power rows to industry (by subtracting power)
df_merge = df_nl_temp.merge(cbs_df[['demand','year','month']], on=['year','month'],how='left')
df_merge.replace(np.nan,0,inplace=True)
df_merge['demand'] = df_merge['demand_x'] - df_merge['demand_y']
df_merge['source'] = "entsog, CBS"

df_merge = df_merge[['country', 'type', 'source', 'year', 'month', 'demand']].copy()
cbs_df = cbs_df[['country', 'type', 'source', 'year', 'month', 'demand']].copy()

df = pd.concat([df, df_merge], ignore_index=True)
df = pd.concat([df, cbs_df], ignore_index=True)

In [51]:
# 5. Apply filtering conditions

conditions_df = pd.DataFrame(filter_conditions_monthly, columns=['country', 'type', 'source'])
filtered_df = df.merge(conditions_df, on=['country', 'type', 'source'])

print("\nAfter applying filters:")
display(filtered_df.head())


After applying filters:


,country,type,source,year,month,demand
0,AT,power,energy-charts,2018,12,0.002566
1,AT,power,energy-charts,2019,1,3.147638
2,AT,power,energy-charts,2019,2,2.488796
3,AT,power,energy-charts,2019,3,1.331403
4,AT,power,energy-charts,2019,4,1.055020


In [52]:
# 6. Aggregate data
aggregated_df = (
    filtered_df.groupby(['country', 'type', 'year', 'month'], as_index=False)
    .agg({
        'demand': 'sum',
        'source': lambda x: ', '.join(sorted(set(x))) if len(set(x)) > 1 else x.iloc[0]
    })
)

In [53]:
def track_available_demand(df):
    pivot_df = df.pivot_table(index=["country", "year", "month"], columns="type", values="demand", aggfunc="sum")
    
    records = []
    
    for index, row in pivot_df.iterrows():
        country, year, month = index
        existing_types = set(row.dropna().index)
        
        records.append({
            "country": country,
            "year": year,
            "month": month,
            "available_types": list(existing_types)
        })
    
    tracking_df = pd.DataFrame(records)
    return tracking_df


def adjust_demand(aggregated_df, new_type, operation, required_types_in=None, required_types_not_in=None):
    """
    param: 
    return:
    """
    tracking_available_demand = track_available_demand(aggregated_df)
    
    required_types_in = required_types_in if required_types_in is not None else set()
    required_types_not_in = required_types_not_in if required_types_not_in is not None else set()
    
    valid_entries = tracking_available_demand[
        tracking_available_demand['available_types'].apply(
            lambda x: required_types_in.issubset(set(x)) and required_types_not_in.isdisjoint(set(x))
        )
    ]
    
    new_rows = []
    modified_entries = set()
    
    for _, row in valid_entries.iterrows():
        country, year, month = row['country'], row['year'], row['month']
        subset = aggregated_df[(aggregated_df['country'] == country) &
                               (aggregated_df['year'] == year) &
                               (aggregated_df['month'] == month)]
        
        new_demand = operation(subset)
        if new_demand is not None:
            new_rows.append({
                'country': country,
                'year': year,
                'month': month,
                'type': new_type,
                'demand': new_demand,
                'source': 'calculated'
            })
            modified_entries.add((country, year, month))  # Track modified entries
    
    if new_rows:
        new_df = pd.DataFrame(new_rows)
        aggregated_df = pd.concat([aggregated_df, new_df], ignore_index=True)
    
    return aggregated_df, modified_entries

def industry_str_demand_operation(subset):
    industry_power_demand = subset[subset['type'] == 'industry-power']['demand'].values
    power_demand = subset[subset['type'] == 'power']['demand'].values
    return industry_power_demand[0] - power_demand[0] if len(industry_power_demand) > 0 and len(power_demand) > 0 else None


def industry_sub_demand_operation(subset):
    total_demand = subset[subset['type'] == 'total']['demand'].values
    power_demand = subset[subset['type'] == 'power']['demand'].values
    household_demand = subset[subset['type'] == 'household']['demand'].values
    return total_demand[0] - power_demand[0] - household_demand[0] if len(total_demand) > 0 and len(power_demand) > 0 and len(household_demand) > 0 else None


def total_demand_operation(subset):
    return subset['demand'].sum()


def industry_household_demand_operation(subset):
    total_demand = subset[subset['type'] == 'total']['demand'].values
    power_demand = subset[subset['type'] == 'power']['demand'].values
    return total_demand[0] - power_demand[0] if len(total_demand) > 0 and len(power_demand) > 0 else None

# Apply operations dynamically based on missing data
updated_aggregated_df, modified_industry_str = adjust_demand(
    aggregated_df, 'industry', industry_sub_demand_operation, 
    {'total', 'power', 'household'}, {'industry'}
)

updated_aggregated_df, modified_industry_str = adjust_demand(
    updated_aggregated_df, 'industry', industry_str_demand_operation, 
    {'industry-power', 'power', 'household'}, {'industry'}
)

# Remove 'industry-power' only for modified entries
updated_aggregated_df = updated_aggregated_df[
    ~((updated_aggregated_df['type'] == 'industry-power') & 
      (updated_aggregated_df[['country', 'year', 'month']].apply(tuple, axis=1).isin(modified_industry_str))
    )
].reset_index(drop=True)

updated_aggregated_df, _ = adjust_demand(
    updated_aggregated_df, 'total', total_demand_operation, None, {'total'}
)


# Remove 'industry' only for modified entries
final_aggregated_df, modified_industry_household = adjust_demand(
    updated_aggregated_df, 'industry-household', industry_household_demand_operation, 
    {'power', 'total'}, {'industry', 'household'}
)


# Recalculate tracking_available_demand
tracking_available_demand = track_available_demand(final_aggregated_df)

tracking_available_demand[tracking_available_demand['country'] == 'FR']

,country,year,month,available_types
915,FR,2015,1,"[power, household, total, industry]"
916,FR,2015,2,"[power, household, total, industry]"
917,FR,2015,3,"[power, household, total, industry]"
918,FR,2015,4,"[power, household, total, industry]"
919,FR,2015,5,"[power, household, total, industry]"
...,...,...,...,...
1043,FR,2025,9,"[power, household, total, industry]"
1044,FR,2025,10,"[power, household, total, industry]"
1045,FR,2025,11,"[power, household, total, industry]"
1046,FR,2025,12,"[power, household, total, industry]"


In [54]:
# Remove unwanted countries
final_aggregated_df = final_aggregated_df[~final_aggregated_df['country'].isin(["EU27_2020", "CY", "MT"])]

def compute_country_contributions(group):
    total_demand = group['demand'].sum()  # Compute total demand for the group
    
    contributions = {
        country: round(float(group[group['country'] == country]['demand'].sum() / total_demand) * 100, 10)  
        for country in group['country'].unique()
    }
    
    return pd.Series({'country_contributions': contributions, 'demand': total_demand})  # Rename total_demand to demand

# Group by type, year, and month and compute demand + country-wise contributions
df_eu = final_aggregated_df[final_aggregated_df['country'] != "UK"].copy()
df_pivot = df_eu.groupby(['type', 'year', 'month']).apply(compute_country_contributions).reset_index()

# Assign 'EU' as the country
df_pivot['country'] = 'EU'

# Add source column
df_pivot['source'] = 'calculated'

# Convert country contributions to a properly formatted dictionary with float values
df_pivot['country_contributions'] = df_pivot['country_contributions'].apply(lambda x: {k: float(v) for k, v in x.items()})

# Concatenate with the original dataset
final_aggregated_df = pd.concat([final_aggregated_df, df_pivot], ignore_index=True)

/var/folders/2d/7gfcftkd69s2z741sq98f0sc0000gn/T/ipykernel_19283/551365456.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_pivot = df_eu.groupby(['type', 'year', 'month']).apply(compute_country_contributions).reset_index()


In [55]:
final_aggregated_df.tail()

,country,type,year,month,demand,source,country_contributions
8333,EU,total,2025,9,200.623396,calculated,"{'AT': 1.8818455059, 'BG': 0.9043242718, 'CZ':..."
8334,EU,total,2025,10,282.893485,calculated,"{'AT': 2.4538195689, 'BG': 0.8180506446, 'CZ':..."
8335,EU,total,2025,11,365.051780,calculated,"{'AT': 2.5193156724, 'BG': 0.7402943269, 'CZ':..."
8336,EU,total,2025,12,400.145731,calculated,"{'AT': 2.6600161202, 'BG': 0.8146294387, 'DE':..."
8337,EU,total,2026,1,188.176321,calculated,"{'AT': 2.4095073814, 'BG': 0.708797731, 'DE': ..."


In [ ]:
# 9. Final processing and export
final_aggregated_df = final_aggregated_df[final_aggregated_df['year'] >= 2019]
final_aggregated_df['demand'] = final_aggregated_df['demand'].round(2)

# # Filter until March 2025
# final_aggregated_df = final_aggregated_df[
#     ((final_aggregated_df['year'] < 2025) |
#      ((final_aggregated_df['year'] == 2025) & (final_aggregated_df['month'] < 12)))
# ]

cutoff_year = 2025
cutoff_month = 11

final_aggregated_df = final_aggregated_df[
    (final_aggregated_df['year'] < cutoff_year) |
    (
        (final_aggregated_df['year'] == cutoff_year) &
        (final_aggregated_df['month'] <= cutoff_month)
    )
]


In [57]:
# Save the results
final_aggregated_df.to_csv("src/data/analyzed/monthly_demand_clean.csv", index=False)
final_aggregated_df.to_json("src/data/analyzed/monthly_demand_clean.json", orient='records', indent=4)
with pd.ExcelWriter("src/data/analyzed/monthly_demand_clean.xlsx", engine='openpyxl') as writer:
    final_aggregated_df.to_excel(writer, sheet_name="Aggregated Data", index=False)

In [58]:
#10. compare totals with eurostat
df_check = final_aggregated_df[['country','demand','type','source','month', 'year']].copy()
df_check = df_check[df_check['type'] == 'total'].copy()
df_check['source'] = 'bruegel'

eurostat_check = eurostat_df.copy()
eurostat_check.loc[eurostat_check['country'] == 'EU27_2020', 'country'] = 'EU'

df_check = pd.concat([df_check, eurostat_check])
df_check = df_check[df_check['year'] > 2018].copy()
df_pivot = df_check.pivot_table(index=['country', 'year', 'month'], columns='source', values='demand')

def plot_and_save_countries(df, output_dir="src/figures/eurostat_totals"):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Loop over each country in the DataFrame index
    for country in df.index.get_level_values(0).unique():
        country_df = df.loc[country]  # Select the country
        country_df = country_df.reset_index()  # Make 'year' and 'month' columns

        # Create a combined 'time' column
        country_df['time'] = country_df['year'].astype(str) + '-' + country_df['month'].astype(str)
        
        # Plot
        plt.figure(figsize=(10, 5))
        plt.plot(country_df['time'], country_df['bruegel'], label='Bruegel', marker='o')
        plt.plot(country_df['time'], country_df['eurostat'], label='Eurostat', marker='s')
        
        # Formatting
        plt.xlabel('')
        plt.ylabel('TWh')
        plt.title(f'{country}')

        # Show every 6th tick label
        tick_positions = range(0, len(country_df), 6)  # Every 6th index
        plt.xticks(tick_positions, country_df['time'].iloc[tick_positions], rotation=45)
        
        plt.legend()
        plt.grid(True)
        
        # Save the figure
        plt.savefig(os.path.join(output_dir, f"{country}.png"), dpi=300, bbox_inches='tight')
        plt.close()  # Close the figure to free memory
    
    print(f"Plots saved to {output_dir}")

# Run the function
plot_and_save_countries(df_pivot)

Plots saved to src/figures/eurostat_totals


In [59]:
#11. A CHECK THAT EU NUMBERS MAKE SENSE

df_check_eu = final_aggregated_df[(final_aggregated_df['country'] == 'EU')].pivot_table(index=['year','month'],columns='type',values='demand')
df_check_eu['total_calculated'] = df_check_eu[['household','industry','industry-household','power']].sum(axis=1)
df_check_eu

type        household  industry  industry-household  power   total  \
year month                                                           
2019 1         267.86     99.82              108.81  90.24  566.72   
     2         195.83     86.41               85.38  68.81  436.43   
     3         164.99     88.39               78.32  60.40  392.10   
     4         108.00     79.94               65.96  57.26  311.16   
     5          85.55     80.51               59.09  58.48  283.63   
...               ...       ...                 ...    ...     ...   
2025 7          33.46     54.97               41.26  56.70  186.39   
     8          30.92     51.47               39.84  49.80  172.03   
     9          44.63     59.63               44.81  51.55  200.62   
     10         89.29     64.59               61.82  67.19  282.89   
     11        145.55     70.01               72.13  77.36  365.05   

type        total_calculated  
year month                    
2019 1                566.73  
     2                436.43  
     3                392.10  
     4                311.16  
     5                283.63  
...                      ...  
2025 7                186.39  
     8                172.03  
     9                200.62  
     10               282.89  
     11               365.05  

[83 rows x 6 columns]

In [60]:
# 12. Calculate annual changes
df_pivot = final_aggregated_df[final_aggregated_df['type'] == 'total'].pivot_table(
    index='country', 
    columns='year', 
    values='demand',
    aggfunc='sum')

df_pivot['AVG'] = df_pivot[[2019,2020,2021]].mean(axis=1)
df_pivot['2022-delta'] = df_pivot[2022] - df_pivot['AVG']
df_pivot['2023-delta'] = df_pivot[2023] - df_pivot['AVG']
df_pivot['2024-delta'] = df_pivot[2024] - df_pivot['AVG']
df_pivot['avg-delta'] = (df_pivot['2022-delta'] + df_pivot['2023-delta'] + df_pivot['2024-delta'])/3

# Add sum row excluding 'EU'
sum_row = df_pivot[df_pivot.index != 'EU'].sum()
df_pivot.loc['TOTAL'] = sum_row

#calculate percentage changes against AVG column
df_pivot['2022-pct-change'] = df_pivot['2022-delta'] / df_pivot['AVG']
df_pivot['2023-pct-change'] = df_pivot['2023-delta'] / df_pivot['AVG']
df_pivot['2024-pct-change'] = df_pivot['2024-delta'] / df_pivot['AVG']
df_pivot['avg-pct-change'] = df_pivot['avg-delta'] / df_pivot['AVG']

#output map data
df_map = df_pivot.copy()
df_map.loc['GB'] = df_map.loc['UK']
df_map = df_map[df_map.index != 'UK']
df_map = df_map[df_map.index != 'EU']
df_map = df_map[df_map.index != 'TOTAL']
df_map['avg-pct-change'] = df_map['avg-pct-change'].round(2)*100
df_map['avg-pct-change'].to_excel("src/data/analyzed/map_data.xlsx")